AISV_801 Natural Language Processing

Instructor Joseph Meyer

Lab5: Train a BERT model to perform text classification on the imdb dataset


Bobby Wen


### Load IMDB dataset from huggingface ###

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
imdb_dataset = load_dataset("imdb")

In [ ]:
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

### Explore data set to check for load errors ###

In [ ]:
imdb_dataset.shape

{'train': (25000, 2), 'test': (25000, 2), 'unsupervised': (50000, 2)}

In [ ]:
imdb_dataset['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

### Define Pre-trained BERT and classifier pre-trained model, and processor optimization ###

In [ ]:
!pip install transformers

In [ ]:
import torch
import transformers
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, Trainer, TrainingArguments

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
# model_name = 'bert-large-uncased'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

### check model states and dictionary ###

In [ ]:
# Print optimizer's state_dict
print("model's state_dict:")
for var_name in model.state_dict():
    print(var_name, "\t", model.state_dict()[var_name])

Streaming output truncated to the last 5000 lines.
        -1.3758e-02,  1.2699e-02, -3.0979e-03,  1.1826e-03,  1.3063e-02,
        -1.9296e-02, -1.4987e-03,  1.5602e-02, -4.0193e-03, -1.6374e-02,
         3.6784e-03, -8.1125e-04,  1.2639e-02, -3.3293e-02,  2.7097e-02,
        -2.9319e-03, -2.4749e-03, -5.6019e-03, -1.8659e-02,  1.9774e-02,
        -6.0444e-03,  1.0730e-03, -8.9020e-03, -2.4519e-02, -2.5099e-03,
         2.0677e-02,  8.0127e-03, -1.4672e-02,  1.3346e-02, -1.2279e-03,
        -7.8777e-03, -3.3122e-03,  1.4286e-02, -1.3947e-02, -4.0466e-03,
        -1.1798e-03, -2.1179e-02,  1.5195e-02,  1.5099e-02,  2.6390e-02,
         1.9045e-02,  7.4747e-02, -8.9122e-03, -3.7042e-02, -2.7115e-02,
        -1.9907e-02, -1.1937e-02,  2.2061e-02,  1.7673e-02, -5.9050e-03,
        -1.6134e-02, -2.8610e-02, -3.0203e-02, -2.8072e-02,  4.6289e-04,
         4.6101e-02, -4.6684e-03, -1.4187e-03, -1.9440e-02, -4.0631e-04,
        -2.7333e-03,  3.6234e-03,  2.4783e-02, -1.0866e-03,  6.2653e-02,


In [ ]:
### Model state to check for __dict__
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

print()

Model's state_dict:
bert.embeddings.word_embeddings.weight 	 torch.Size([30522, 768])
bert.embeddings.position_embeddings.weight 	 torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight 	 torch.Size([2, 768])
bert.embeddings.LayerNorm.weight 	 torch.Size([768])
bert.embeddings.LayerNorm.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight 	 torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight 	 torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight 	 torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight 	 torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight 	 torch.Size([768])
bert.encoder.layer.0.attention.outp

### Define computer metrics for used during training ###

In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from sklearn.metrics import f1_score

def compute_f1(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return f1_score(labels, preds, average='weighted')

### Define tokenization parameters ###

In [ ]:
def tokenize(examples):
    outputs = tokenizer(examples['text'], truncation=True)
    return outputs

tokenized_imdb_dataset = imdb_dataset.map(tokenize, batched=True)

### Define training argument and parameters for training model ###

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
# train_dataset = torch.utils.data.TensorDataset(
#     torch.tensor(train_encodings.input_ids),
#     torch.tensor(train_encodings.attention_mask),
#     torch.tensor(train_labels)
# )

# val_dataset = torch.utils.data.TensorDataset(
#     torch.tensor(val_encodings.input_ids),
#     torch.tensor(val_encodings.attention_mask),
#     torch.tensor(val_labels)
# )

training_args = TrainingArguments(num_train_epochs=1,
                                  output_dir="./bert_classifier",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  logging_dir="./logs",
                                  logging_steps=500,
                                  save_steps=2000,
                                  evaluation_strategy="steps",
                                  #evaluation_strategy="epoch",
                                  eval_steps=500,
                                  save_total_limit=2,
                                 )

trainer = Trainer(model=model,
                  tokenizer=tokenizer,
                  data_collator=data_collator,
                  args=training_args,
                  train_dataset=tokenized_imdb_dataset["train"],
                  eval_dataset=tokenized_imdb_dataset["test"],
                  compute_metrics=compute_metrics
                  #compute_metrics=compute_f1
                 )

### Train model.  This takes a long time ###

In [ ]:
# # Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.309700,0.207720,0.918520
1000,0.236500,0.185264,0.932120
1500,0.200200,0.172885,0.937920


<ipython-input-10-173119baf1a7>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


TrainOutput(global_step=1563, training_loss=0.24512116693947, metrics={'train_runtime': 1535.0016, 'train_samples_per_second': 16.287, 'train_steps_per_second': 1.018, 'total_flos': 6524282617557120.0, 'train_loss': 0.24512116693947, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.17291691899299622,
 'eval_accuracy': 0.93872,
 'eval_runtime': 267.4479,
 'eval_samples_per_second': 93.476,
 'eval_steps_per_second': 5.844,
 'epoch': 1.0}